In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
artists = pd.read_parquet('https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/data/Artist.parquet.gzip')
artists.head()

,id,name,url,summary,picture,birthplace,deathplace,birthdate,deathdate,cause_of_death
0,0,Vincent Van Gogh,http://wikigallery.org/wiki/artist36933/Vincen...,Vincent Willem van Gogh (Dutch: [ˈvɪnsɛnt ˈʋɪl...,0.0,0.0,342.0,1853-03-30,1890-07-29,Gunshot wound
1,1,Pierre Auguste Renoir,http://wikigallery.org/wiki/artist39254/Pierre...,Pierre-Auguste Renoir (French: [pjɛʁ oɡyst ʁən...,1.0,1.0,343.0,1841-02-25,1919-12-03,None
2,2,Claude Oscar Monet,http://wikigallery.org/wiki/artist39249/Claude...,"Oscar-Claude Monet (UK: , US: , French: [klod ...",2.0,2.0,344.0,1840-11-14,1926-12-05,None
3,3,Fernando Botero,http://wikigallery.org/wiki/artist37052/Fernan...,Fernando Botero Angulo (born 19 April 1932) is...,3.0,3.0,NaN,1932-04-19,None,None
4,4,Jean-Léon Gérôme,http://wikigallery.org/wiki/artist46453/Jean-L...,Jean-Léon Gérôme (11 May 1824 – 10 January 190...,4.0,4.0,2.0,1824-05-11,1904-01-10,None


### Artist
The 'Artist' table is a central component of the dataset and provides information on famous painters and their careers.

Next you'll find some basic demographics about the artists in this dataset.

In [21]:
# Select all artists that have death date as NaN
artists_alive = artists[artists['deathdate'].isna()]
print(artists_alive.shape)
# Select all artists that have 'deathdate' different from NaN
artists_dead = artists[artists['deathdate'].notna()]
print(artists_dead.shape)

(346, 10)
(270, 10)
